# Libs

In [1]:
import os
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

c:\Users\Admin\Desktop\DANG NHI\repos\master-llm-rag-vnlaw


In [2]:
import ollama
import asyncio
from ollama import AsyncClient
import numpy as np

from langchain.vectorstores import Chroma

import chromadb
from chromadb import Settings, EmbeddingFunction, Embeddings

from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

from nltk.tokenize import word_tokenize

# Load embedding

In [3]:
base_model_id = "FacebookAI/xlm-roberta-base"
trained_model_path = "./models/trained_embedding_small_data/"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 50

import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModel.from_pretrained(trained_model_path)
model.eval()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
model.to(device)

print("Model ready !!!")

c:\Users\Admin\miniconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model ready !!!


In [4]:
context = "Tôi đang viết trên bảng ở trường."
token = tokenizer(context, return_tensors=None, truncation=True, max_length=MAX_LEN)
tokens_text = tokenizer.convert_ids_to_tokens(token["input_ids"])
print(tokens_text)

['<s>', '▁Tôi', '▁đang', '▁viết', '▁trên', '▁bảng', '▁ở', '▁trường', '.', '</s>']


In [16]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model, tokenizer, max_length=512):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length


    def embed_documents(self, texts):
        inputs = self.tokenizer(texts, 
                                padding='max_length', 
                                max_length=self.max_length, 
                                return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
    
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
        return embeddings.cpu().numpy().tolist()[0]

myembed = MyEmbeddingFunction(model, tokenizer)

In [ ]:
# text = 'Tôi đi học'
# with torch.no_grad():
#     token = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=MAX_LEN)
#     output = model(**token)

# output = myembed.embed_documents(["Tôi đi học"])
output1 = myembed.embed_query("Tôi đang viết trên bảng ở trường")

# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# similarity = cosine_similarity(output, output1)
# print(f"Cosine similarity:\n {similarity}")

# Vector database và Embedding model

In [7]:
# client = chromadb.PersistentClient(
#     path="./data/chroma_db/",
#     settings=Settings(allow_reset=False,),)
# client.list_collections()

# collection = client.get_collection(name="VNLaws", embedding_function=myembed)
# print(collection.count())

# collection.query(
#     query_texts = ["Tôi có thể làm gì với hợp đồng lao động?"],
#     n_results=10,
#     where_document={"$contains": "Tôi có thể làm gì với hợp đồng lao động?"},
# )

In [21]:
db = Chroma(
    embedding_function=myembed, # lấy embedding model đã train được
    persist_directory='./chroma_db',
    collection_name="VNLaws",
    )
vectorstore_retriever = db.as_retriever(search_kwargs={"k": 10})

stored_data = db._collection.get()
zip_doc = list(zip(stored_data["documents"], stored_data["metadatas"]))
docs = [Document(page_content=doc, metadata=metadata) for doc,metadata in zip_doc]

In [22]:
keyword_retriever = BM25Retriever.from_documents(docs, search_kwargs={"k": 10})
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retriever, keyword_retriever], weights=[0.5, 0.5])

In [25]:
relevants = ensemble_retriever.get_relevant_documents("Tôi có thể làm gì với hợp đồng lao động trong công an nhân dân?")
relevants

[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 13. Nội dung hợp đồng lao động\n1. Hợp đồng lao động phải có những nội dung chủ yếu theo quy định tại khoản 1 Điều 21 Bộ luật Lao động .\n2. Người lao động làm việc ở các đơn vị, bộ phận trọng yếu, cơ mật trong Công an nhân dân, trước khi ký kết hợp đồng với người lao động, đơn vị trực tiếp sử dụng phải yêu cầu người lao động cam kết bằng văn bản (theo mẫu ban hành kèm theo Thông tư này) về công tác bảo mật trong thời gian làm việc và sau khi kết thúc hợp đồng lao động; đồng thời triển khai các biện pháp phù hợp, trang bị kiến thức cần thiết cho người lao động về việc bảo vệ bí mật Nhà nước.\n3. Trong trường hợp cần quy định chi tiết hoặc để sửa đổi, bổ sung một số điều, khoản của hợp đồng lao động, người sử dụng lao động và người lao động có thể l

In [39]:
from collections import Counter

name = [i.metadata["name"] for i in relevants]
name_counts = Counter(name)
ranked_names = sorted(name_counts.items(), key=lambda x: x[1], reverse=True)

sorted_documents = []
for name, count in ranked_names:
    for doc in relevants:
        if doc.metadata["name"] == name:
            sorted_documents.append(doc)

top_documents = sorted_documents[:10]
# sorted_doc = [doc for doc in relevants for name, _ in ranked_names if doc.metadata["name"] == name]
# while len(top_documents) < max_chunks:
#     for name, count in ranked_names:
#         doc
#         top_documents.append([doc for doc in ])

In [40]:
top_documents

[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 13. Nội dung hợp đồng lao động\n1. Hợp đồng lao động phải có những nội dung chủ yếu theo quy định tại khoản 1 Điều 21 Bộ luật Lao động .\n2. Người lao động làm việc ở các đơn vị, bộ phận trọng yếu, cơ mật trong Công an nhân dân, trước khi ký kết hợp đồng với người lao động, đơn vị trực tiếp sử dụng phải yêu cầu người lao động cam kết bằng văn bản (theo mẫu ban hành kèm theo Thông tư này) về công tác bảo mật trong thời gian làm việc và sau khi kết thúc hợp đồng lao động; đồng thời triển khai các biện pháp phù hợp, trang bị kiến thức cần thiết cho người lao động về việc bảo vệ bí mật Nhà nước.\n3. Trong trường hợp cần quy định chi tiết hoặc để sửa đổi, bổ sung một số điều, khoản của hợp đồng lao động, người sử dụng lao động và người lao động có thể l

In [34]:
for name, _ in ranked_names:
    print(name)

Thông tư 22/2022/TT-BCA
Quyết định 641/QĐ-UBND
Bản án 04/2022/LĐ-PT
Hướng dẫn 33/HD-VKSTC


In [33]:
sorted_doc

[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 13. Nội dung hợp đồng lao động\n1. Hợp đồng lao động phải có những nội dung chủ yếu theo quy định tại khoản 1 Điều 21 Bộ luật Lao động .\n2. Người lao động làm việc ở các đơn vị, bộ phận trọng yếu, cơ mật trong Công an nhân dân, trước khi ký kết hợp đồng với người lao động, đơn vị trực tiếp sử dụng phải yêu cầu người lao động cam kết bằng văn bản (theo mẫu ban hành kèm theo Thông tư này) về công tác bảo mật trong thời gian làm việc và sau khi kết thúc hợp đồng lao động; đồng thời triển khai các biện pháp phù hợp, trang bị kiến thức cần thiết cho người lao động về việc bảo vệ bí mật Nhà nước.\n3. Trong trường hợp cần quy định chi tiết hoặc để sửa đổi, bổ sung một số điều, khoản của hợp đồng lao động, người sử dụng lao động và người lao động có thể l

In [31]:
name_counts

Counter({'Thông tư 22/2022/TT-BCA': 9,
         'Quyết định 641/QĐ-UBND': 3,
         'Bản án 04/2022/LĐ-PT': 1,
         'Hướng dẫn 33/HD-VKSTC': 1})

# Test full text search

C:\Users\Admin\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [17:30<00:00, 79.2kiB/s]   


InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 768

# Create ollama client

In [30]:
ollama_api_url = "https://6efb-34-141-226-213.ngrok-free.app" # Thay đổi mỗi lần host

ollama_client = ollama.Client(
    host=ollama_api_url,
    headers = {'Header': 'application/json'}
)

In [26]:
system_prompt = "Xây dựng chương trình RAG với chatbot trung thực.\n" \
"Tôi có bộ dữ liệu gồm các Lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Lao Động, Nhà Đất và Tài Chính Ngân Hàng. " \
"Trong đó bao bồm các văn bản hành chính của Chính phủ.\n"

agent_judgment1 = 'Bạn là một agent dùng để đánh giá câu trả lời. ' \
'Với câu hỏi "{question}", Bạn có thể trả lời câu hỏi bằng cơ sở dữ liệu của tôi được hay không (trả lời 1 nếu có thể, trả lời 0 nếu không thế).' \
'Trả lời:' \

agent_search_plan = 'Bạn là một agent để dự đoán thông tin cần truy vấn.\n' \
'Theo bạn để trả lời câu hỏi "{question}", thì cần những thông tin gì và đến từ Lĩnh vực nào?\n' \
'Trả lời tối đa 3 dòng và theo cú pháp "<linh_vuc>: <thong_tin_can>", linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trùng Lĩnh vực nhưng khác nhau thông tin cần.\n' \
'Trả lời: '

agent_judgment2 = 'Bạn là một agent reranking' \

answer_prompt_template = \
    """
    Bạn là một chuyên gia trả lời câu hỏi về luật về các lĩnh vực bao gồm Bảo Hiểm, Lao Động, Nhà Đất và Tài Chính Ngân Hàng và câu trả lời dựa trên thông tin được cung cấp. Hãy trả lời câu hỏi ngắn gọn và dễ hiểu. Nếu không có thông tin để trả lời câu hỏi, hãy gợi ý câu hỏi khác. Đây là thông tin bạn có được cho câu hỏi này: {context} và chủ đề của câu hỏi {plans}. Dựa vào thông tin có được hãy trả lời câu hỏi sau đây: {question} \n
    Trả lời: 
    """



### Đặt câu hỏi

In [36]:
# question = "Quần đảo Trường Sa thuộc tỉnh nào?"
question = "Tôi có thể làm gì với hợp đồng lao động?"

Đánh giá khả năng trả lời câu hỏi

In [ ]:
# judment1: Trả lời được hay không
response = ollama_client.chat(
    model="gemma3:12b",
    messages=[{"role": "user", "content": system_prompt + agent_judgment1.format(question=question)}],
    stream=False,
)

print(response.message.content)

Lựa chọn thông tin

In [32]:
def parse_judgment(response_text: str) -> int:
    for char in response_text.strip():
        if char in ["0", "1"]:
            return int(char)
    return 0


if parse_judgment(response.message.content) == 1:
    print("Có thể trả lời được")

    response = ollama_client.chat(
        model="gemma3:12b",
        messages=[
            {
                "role": "user",
                "content": system_prompt + agent_search_plan.format(question=question),
            }
        ],
        stream=False,
    )
    plans = response.message.content.split("\n")


else:
    print("Không thể trả lời được")

Có thể trả lời được


In [ ]:
# PROMPT = agent_search_plan.format(question=question)

def answer(question, plans):
    if isinstance(plans, list):
        plans = ", ".join(plans)

    # embed the query
    query_embed = myembed.embed_query( question + " " + plans)
    try:
        context_list = collection.query(
            query_embeddings=query_embed[:768], n_results=5
        )["documents"][0]
        context = " ".join(context_list)
    except:
        try:
            context_list = collection.query(
                query_embeddings=query_embed[0][:768], n_results=5
            )["documents"][0]
            context = " ".join(context_list)
        except:
            context = "Không tìm thấy thông tin."

    answer_prompt = system_prompt + answer_prompt_template.format(
        question=question, context=context, plans=plans
    )

    response = ollama_client.chat(
        model="gemma3:12b",
        messages=[
            {
                "role": "user",
                "content": answer_prompt,
            }
        ],
        stream=False,
    )
    return response.message.content

In [ ]:
answer(question, plans)